In [1]:
# %load ../firstcell.py
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
model_fp32 = './vgg16-12.onnx'

In [3]:
import deepcrunch
deepcrunch.config(framework="onnx", mode="inference", backend="onnx")

In [7]:
quantized_model = deepcrunch.quantize(model_fp32)

In [5]:
deepcrunch.save(quantized_model, output_path="vgg16_quantized.onnx")

Quantized model saved to vgg16_quantized.onnx


In [6]:
original_model_size = deepcrunch.performance.size_in_mb("./vgg16-12.onnx", human_readable=True)

quantized_model_size = deepcrunch.performance.size_in_mb("./vgg16_quantized.onnx", human_readable=True)

print(f"Original model size: {original_model_size}")
print(f"Quantized model size: {quantized_model_size}")

Original model size: 527.80 MB
Quantized model size: 132.01 MB
